In [ ]:
import pandas as pd

columns = [
    "date_time", "city", "state", "country", "shape", "duration_seconds",
    "duration_hours_mins", "comments", "date_posted", "latitude", "longitude"
]

url = "https://github.com/UIUC-iSchool-DataViz/is445_data/raw/main/ufo-scrubbed-geocoded-time-standardized-00.csv"
df = pd.read_csv(url, header=None, names=columns)

print(df.head())

In [ ]:
df['date_time'] = pd.to_datetime(df['date_time'], errors='coerce')
df = df.dropna(subset=['latitude', 'longitude', 'date_time'])

df['year'] = df['date_time'].dt.year
sightings_by_year = df.groupby('year').size().reset_index(name='count')

print(sightings_by_year.head())

In [ ]:
import altair as alt

alt.data_transformers.disable_max_rows()

chart1 = alt.Chart(sightings_by_year).mark_line(point=True).encode(
    x=alt.X('year:O', title='Year'),
    y=alt.Y('count:Q', title='Number of UFO Sightings'),
    tooltip=['year', 'count']
).properties(
    title='UFO Sightings Over Time',
    width=600,
    height=400
)

chart1

In [ ]:
import altair as alt

filtered_df = df[df['country'].notna() & df['latitude'].notna() & df['longitude'].notna()]

dropdown = alt.binding_select(
    options=sorted(filtered_df['country'].unique()),
    name='Select Country: '
)

country_param = alt.param(
    name='country_param',
    bind=dropdown,
    value='us' 
)

chart2 = alt.Chart(filtered_df).mark_circle(size=60).encode(
    longitude='longitude:Q',
    latitude='latitude:Q',
    color=alt.Color('shape:N', title='UFO Shape'),
    tooltip=['city:N', 'state:N', 'shape:N', 'date_time:T']
).add_params(
    country_param
).transform_filter(
    alt.datum.country == country_param  
).properties(
    width=750,
    height=450,
    title='Interactive Map of UFO Sightings by Country'
)

chart2


In [5]:
chart1.save('chart1_ufo_time.html')
chart2.save('chart2_ufo_map.html')